This notebook contains the code for a simple Sudoku Solver application built using the Tkinter library in Python.

# Import needed Libraries

If you are importing ipynb file. Make sure to install nbimporter using command "pip install nbimporter" before using this nbimporter to import other module.
Tkinter is used in this program for graphic user interface

In [25]:
import tkinter as tk
from PIL import Image, ImageTk
import random
import time
import nbimporter 
from SudokuBackend import SudokuSolver,CreateSudoku,ValidateSudoku,SudokuDifficulty,board_quest

# User interface

The user interface is created using Tkinter widgets. Below is the code that sets up the main window, labels, and other UI elements.

In [26]:
root=tk.Tk()
root.title("Sudoku Solver")
root.geometry("390x560")
root.configure(bg="white")
label = tk.Label(root, text="Fill in the numbers and click solve",bg="white")
label.grid(row=0,column=1,columnspan=10)

errLabel = tk.Label(root, text="",fg="red",bg="white")
errLabel.grid(row=15,column=5,rowspan=2,columnspan=10)

solvedLabel = tk.Label(root, text="",fg="green",bg="white")
solvedLabel.grid(row=16,column=5,columnspan=10)

hintLabel = tk.Label(root, text="",fg="blue",bg="white")
hintLabel.grid(row=15,column=1,columnspan=5)

TimerLabel = tk.Label(root, text="Time: 00:00:00",bg="white")
TimerLabel.grid(row=16,column=1,columnspan=5,pady=10)

# Sudoku Solver Initialization

In this section, we initialize the core variables that will be used throughout the Sudoku solver application. These variables include the storage for the puzzle cells, the initial state of the puzzle, an undo stack for user actions, the difficulty level of the puzzle, and the variables required for the timer functionality.

## Core Variables

In [27]:

cells={} # A dictionary to store the current state of each cell in the Sudoku grid.
initial_state={} # A dictionary to keep track of the initial values of the cells when the puzzle is first loaded or generated.
undo_stack=[] # A list that functions as a stack to keep track of user actions for the undo feature.
difficulty_level=None #Initiating difficulty level


## Timer Variables

In [28]:
# Variables to keep track of the timer
start_time = None # A variable to store the starting time when the timer is activated.
timer_running = False # A boolean flag to indicate whether the timer is currently running.


# Solving the Sudoku Puzzle

## solve_gui Function

The `solve_gui` function is triggered when the user requests to solve the puzzle. It performs the following steps:
1. Retrieves the current values from the GUI board using the `getValues` function.
2. Initializes a `SudokuSolver` object with the current board.
3. Validates the initial Sudoku board using the `ValidateSudoku` class.
4. If the board is valid, it attempts to solve the puzzle.
5. Stops the timer if the puzzle is solved.
6. Updates the GUI with the solved board using the `update_gui` function.
7. Displays a success message to the user.
8. If no solution is found or the board is invalid, an error message is displayed.

In [29]:
def solve_gui():
    board=getValues() # Retrieves the current values
    sudoku_solver=SudokuSolver(board) # Initializes a SudokuSolver object
    sudoku_validate=ValidateSudoku(board) # Validates the initial Sudoku
    if sudoku_validate.validate():
        if sudoku_solver.solve():
            stop_timer()
            update_gui(sudoku_solver.board)
            solvedLabel.configure(text="Sudoku has been completed")
    else:
        errLabel.configure(text="No solution")

## update_gui Function

The `update_gui` function takes the solved board as input and updates each cell in the GUI with the corresponding solution. It iterates over the board and:

1. Accesses each cell in the `cells` dictionary by its grid coordinates.
2. Clears the current value in the cell.
3. Inserts the new value from the solved board.

These functions work together to provide the core functionality of the Sudoku solver application, allowing users to quickly and easily find solutions to their puzzles.

In [30]:
# Fill in the board with solutions
def update_gui(board):
    for i in range(len(board)):
        for j in range (len(board[i])):
            cell = cells[(i+2,j+1)]
            cell.delete(0,"end")
            cell.insert(0,str(board[i][j]))

# Clearing the Sudoku Board

The `clearValues` function is designed to reset the Sudoku board to its initial state. It is typically called when the user wants to start over or clear their input. Here's what the function does:

1. Clears the `undo_stack`, which is used to keep track of user actions for the undo feature.
2. Resets the error and solved messages by clearing the text of `errLabel` and `solvedLabel`.
3. Iterates over each cell in the grid:
   - Retrieves the cell widget from the `cells` dictionary using its grid coordinates.
   - Checks if the current value of the cell differs from the `initial_state` value.
   - If the cell was modified by the user, it clears the cell's content and resets the text color to black.
   - Re-enables validation for the cell to ensure that only valid numbers can be entered after clearing.

In [31]:
def clearValues():
    global undo_stack
    undo_stack.clear()
    errLabel.configure(text="")
    solvedLabel.configure(text="")
    for row in range(2,11):
        for col in range(1,10):
            cell=cells[(row,col)]
            if cell.get() != initial_state[(row, col)]:  # Check if the cell was modified by the user
                cell.delete(0,'end')
                cell.config(fg="blue")
                cell.config(validate="key", validatecommand=(reg, "%P", str(row), str(col))) # Run ValidateNumber function after clear


#  Starting a New Sudoku Game


The `newgame` function is responsible for initializing a new game session. It sets up the board and prepares the game according to the selected difficulty level. Here's a breakdown of the steps performed by this function:

1. Stops the current timer.
2. Enables any disabled buttons on the UI using the `button_enabled` function.
3. Draws a fresh 9x9 Sudoku grid with the `draw9x9Grid` function.
4. Enables all cells for user input.
5. Clears any existing values and resets the text color to black for all cells.
6. Generates a new Sudoku puzzle board with the `board_quest` function, which is then passed to the `CreateSudoku` class along with the current `difficulty_level`.
7. Calls the `board_create` method of the `CreateSudoku` instance to create a new board.
8. If a valid board is generated, populates the grid with the new puzzle using the `populateGridWithBoard` function.
9. If the board is invalid, displays an error message using `errLabel`.

This function is triggered when the user selects to start a new game, ensuring a smooth transition and a clear board ready for a new challenge.

In [32]:
def newgame():
    errLabel.config(text="")
    solvedLabel.config(text="")
    hintLabel.config(text="")
    stop_timer()
    button_enabled()
    draw9x9Grid()
    enable_all_cells()
    for cell in cells.values():
        cell.config(fg="black")
        cell.delete(0,'end')
    board_q=board_quest()
    sudoku_create=CreateSudoku(board_q,difficulty_level)
    board=sudoku_create.board_create()
    if board:
        populateGridWithBoard(board)    
    else:
        errLabel.config(text="Sudoku is invalid!")
        button_disabled()
        readonly_all_cells()
        buttons=[btn_newgame, btneasy, btnmedium, btnhard]
        for button in buttons:
            button.config(state='disabled')

# Undo Functionality

The `undo` function enables players to revert their last move. It works by popping the last action from the `undo_stack`, which records each move's row, column, previous value, and color. The function then updates the corresponding cell with the previous value and color. If the cell was part of the initial puzzle, it is set back to read-only; otherwise, it is made editable. This feature is essential for providing a user-friendly experience, allowing mistakes to be easily corrected.

In [33]:
def undo():
    if undo_stack:
        row, col, previous_value, previous_color = undo_stack.pop()
        cell = cells[(row, col)]
        cell.delete(0, 'end')
        if previous_value: 
            cell.insert(0, previous_value) # If there was a previous value, insert it back
            cell.config(fg=previous_color) # Restore the previous color

        # If the cell is part of the initial puzzle, make it read-only again
        if initial_state.get((row, col)) != "":
            cell.config(state='readonly')
        else:
            cell.config(state='normal')
        cell.config(validate="key", validatecommand=(reg, "%P", str(row), str(col)))



# Providing Hints

In [34]:
def hint():
    board = getValues()  # Retrieves the current values
    sudoku_solver = SudokuSolver(board)  # Initializes a SudokuSolver object
    sudoku_validate = ValidateSudoku(board)  # Validates the initial Sudoku
    if sudoku_validate.validate():
        empty_cells = [(i, j) for i in range(9) for j in range(9) if board[i][j] == 0]
        if empty_cells:  # Check if there are any empty cells left
            sudoku_solver.solve()
            row, col = random.choice(empty_cells)  # Choose a random empty cell
            hint_value = sudoku_solver.board[row][col]  # Get the value from the solved board
            cells[(row+2, col+1)].delete(0, 'end')  # Adjusting indices to match the grid layout
            cells[(row+2, col+1)].insert(0, str(hint_value))  # Insert the hint value
            cells[(row+2, col+1)].config(fg="blue")  # Optional: change the color to indicate a hint
            cells[(row+2, col+1)].config(state="readonly")  # Make the cell read-only
            max_hint()  # Update the hint count
            max_correct()  # Check if the puzzle is completed
    else:
        errLabel.configure(text="No solution")

# Timer

The Sudoku game includes a timer to track how long the player has been solving the current puzzle. The following functions manage the timer:

- `update_timer`: This function updates the timer label every second with the elapsed time since the timer started. It formats the time into hours, minutes, and seconds and schedules itself to be called again after 1 second if the timer is running.

- `start_timer`: This function starts the timer by recording the current time as the start time and setting the `timer_running` flag to `True`. It then calls `update_timer` to initiate the timer updates.

- `stop_timer`: This function stops the timer by setting the `timer_running` flag to `False`.

These functions work together to provide a real-time timer display that enhances the gameplay experience by allowing players to track their progress.


In [35]:
def update_timer():
    if timer_running:
        # Calculate the elapsed time
        elapsed_time = time.time() - start_time
        # Convert the elapsed time into hours, minutes, and seconds
        hours, remainder = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        # Update the timer label
        TimerLabel.config(text="Time: {:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds)))
        # Schedule the update_timer function to be called after 1000ms (1 second)
        root.after(1000, update_timer)

def start_timer():
    global start_time, timer_running
    if not timer_running:
        # Record the start time and set the timer to running
        start_time = time.time()
        timer_running = True
        update_timer()

def stop_timer():
    global timer_running
    timer_running = False

# Draw colorful grid

In [36]:
def draw3x3Grid(row, column, bgcolor):
    for i in range(3):
        for j in range(3):
            grid_row = row + i + 1
            grid_col = column + j + 1
            sv=tk.StringVar()

            e = tk.Entry(root, width=5, bg=bgcolor, justify='center',
                      validate="key", validatecommand=(reg, "%P", str(grid_row), str(grid_col)))
            e.grid(row=grid_row, column=grid_col, sticky="nsew", padx=1, pady=1, ipady=5) #users can enter their answer
            e.bind('<Key>', lambda event, r=grid_row, c=grid_col: key_pressed(event,r,c)) #trigger key_pressed function anytime a key in keyboard is pressed inside the box
            cells[(grid_row, grid_col)] = e
            
def draw9x9Grid():
    color="#D0ffff"
    for rowNo in range(1,10,3):
        for colNo in range(0,9,3):            
            draw3x3Grid(rowNo,colNo,color)
            if color =="#D0ffff": #light cyan color
                 color="#ffffd0" #light yellow color
            else:
                color="#D0ffff"

Validate user's entry

In [37]:
def ValidateNumber(P, row, col):
    row = int(row)  # Convert row to an integer
    col = int(col)  # Convert col to an integer
  
    # Adjust the row and col to match the board's indexing (0-based)
    board_row = row - 2
    board_col = col - 1
    # If the cell is cleared, reset the text color and allow the change
    if P == "":
        undo_stack.append((row, col, cells[(row, col)].get(), cells[((row, col))].cget('fg')))
        cells[((row, col))].delete(0,'end')
        cells[(row, col)].config(fg="black")
        return True
    
    # If the input is a single digit, check if it's valid
    if P.isdigit() and len(P) == 1:
        start_timer()
        num = int(P)
        undo_stack.append((row, col, cells[(row, col)].get(), cells[((row, col))].cget('fg')))
        # Temporarily set the cell to 0 to avoid conflict with itself during validation
        cells[(row, col)].delete(0, "end")
        cells[(row, col)].insert(0, "0")
        
        # Get the current state of the board
        board = getValues()
        sudoku_solver=SudokuSolver(board)
        # Restore the cell's value
        cells[(row, col)].delete(0, "end")
        cells[(row, col)].insert(0, P)
        
        # Check if the number is valid in the current board state
        if sudoku_solver.valid(num, (board_row, board_col)):
            cells[(row, col)].config(fg="green")
            max_correct()
            max_mistakes()
            max_hint()
            return True  # Allow the change
        else:
            cells[(row, col)].config(fg="red")
            max_mistakes()
            max_hint()
            return True
            
    else:
        # If the input is not a single digit, reject the change
        return False

reg=root.register(ValidateNumber)

def key_pressed(event,row,col):
    if event.keysym == "BackSpace" or "Delete":
        current_value=cells[(row,col)].get()
        current_color = cells[(row, col)].cget('fg') #get current color whether red or green
        # If the cell is about to be cleared, reset the text color and push to undo stack
        if len(current_value)==1:
            undo_stack.append((row,col,current_value,current_color))   
    cells[(row,col)].config(validate="key", validatecommand=(reg, "%P", str(row), str(col))) # Run ValidateNumber function after clear

def on_value_change(sv, row, col):
    value = sv.get()
    ValidateNumber(value, row, col)

def getValues():
    board=[]
    errLabel.configure(text="")
    solvedLabel.configure(text="")
    for row in range(2,11):
        rows=[]
        for col in range(1,10):
            val = cells[(row,col)].get()
            if val=="":
                rows.append(0)
            else:
                rows.append(int(val))
        board.append(rows)
    return board

# Count valid/invalid numbers

The Sudoku solver includes functionality to count the number of valid and invalid numbers entered by the player. This can be useful for providing feedback or for implementing game rules that depend on the number of mistakes.

- `count_valid_numbers`: This function iterates through all the cells in the Sudoku grid and counts how many numbers are marked as valid (with a green foreground color). It returns the total count of valid numbers.

- `count_invalid_numbers`: Similarly, this function iterates through all the cells and counts how many numbers are marked as invalid (with a red foreground color). It returns the total count of invalid numbers.

These functions help in monitoring the player's progress and can be used to trigger specific actions based on the game's state.

In [38]:
# Check how many valid numbers
def count_valid_numbers():
    count = 0
    for cell in cells.values():
        if cell.cget('fg') == 'green':
            count += 1
    return count

# Check how many invalid numbers
def count_invalid_numbers():
    count=0
    for cell in cells.values():
        if cell.cget('fg') == 'red':
            count += 1
    return count

## Count Hint Numbers

In [39]:
# Check how many hint
def count_hint_numbers():
    count = 0
    for cell in cells.values():
        if cell.cget('fg') == 'blue':
            count += 1
    return count

## Completion Check Function

- Calls `count_valid_numbers` to get the current count of valid numbers entered by the player.
- Compares the count of valid numbers to the `difficulty_level` to check for completion.
- If the number of valid numbers matches the difficulty level, it updates the `solvedLabel` with a congratulatory message, stops the timer, disables the buttons, and makes all cells read-only.
- This function is crucial for providing a satisfying end to the game and for ensuring that the player receives appropriate feedback upon completion.

The function relies on the assumption that the `difficulty_level` corresponds to the total number of cells that need to be filled in correctly to consider the puzzle solved.

In [40]:
def max_correct(): 
    # Update the valid number count if needed
    valid_count = count_valid_numbers()
    hint_count = count_hint_numbers()
    difficulty_new_level=difficulty_level-hint_count
    if valid_count==difficulty_new_level:
        solvedLabel.config(text="Congratulations! \nYou have completed the sudoku.")
        stop_timer()
        button_disabled()
        readonly_all_cells()

## Allowable Mistake Limit Function

Only allow user to make mistakes up to 3.

The `max_mistakes` function is responsible for tracking the number of mistakes made by the player and providing feedback based on the count of invalid numbers:

- Calls `count_invalid_numbers` to get the current count of invalid numbers (marked with a red foreground color).
- If the number of mistakes is between 1 and 2, it updates the `errLabel` with the current mistake count.
- If the number of mistakes reaches 3 or more, it updates the `errLabel` with a message indicating that the maximum number of mistakes has been reached and prompts the player to start a new game.
- Additionally, when the mistake limit is reached, it disables all cells and buttons to prevent further input.
- The function returns the current count of invalid numbers.

This function enhances the game by setting a limit on the number of allowable mistakes, thus adding an extra layer of challenge for the player.

In [41]:
def max_mistakes(): 
     # Update the invalid number count if needed
    invalid_count = count_invalid_numbers()    
    if 0<invalid_count<3:
        errLabel.config(text=f"Mistakes: {invalid_count}")
    elif invalid_count>=3:
        errLabel.config(text=f"Oops! \nMax mistakes: {invalid_count}")
        readonly_all_cells()
        button_disabled()
        stop_timer()
    return invalid_count

## Hint Limitation

The `max_hint` function is responsible for tracking the number of hints used by the player and updating the hint status label accordingly. It also disables the hint button if the maximum number of hints has been reached.

In [42]:
def max_hint():
    hint_count=count_hint_numbers() # Get the current count of hints used
    hint_left=3-hint_count # Calculate the number of hints left
    if hint_count==3:
        hintLabel.config(text=f"You have used max hints: {hint_count}") # Update the hint label
        btnhint.config(state="disabled") # Disable the hint button
    else:
        hintLabel.config(text=f"Hints left: {hint_left}") # Update the hint label with hints left

## Button and Cell State Management

The following functions are used to control the interactivity of the Sudoku solver's buttons and cell entry fields, enabling or disabling them as needed based on the game's state.

### Button State Functions


- `button_enabled`: This function enables a predefined list of buttons (`btnsolve`, `btnclear`, `btn_undo`) by setting their state to "normal". This allows the user to interact with the buttons when the game is in a state where their functionality is required.

- `button_disabled`: Conversely, this function disables the same list of buttons by setting their state to "disabled". This is used to prevent user interaction with the buttons during certain conditions, such as when the maximum number of mistakes has been reached or the puzzle is solved.

In [43]:
# Button enabled
def button_enabled():
    buttons = [btnsolve,btnclear,btn_undo,btnhint]
    for button in buttons:
        button.config(state="normal")

# Button disabled
def button_disabled():
    buttons = [btnsolve,btnclear,btn_undo,btnhint]
    for button in buttons:
        button.config(state="disabled")

### Cell Entry State Functions

- `enable_all_cells`: This function sets the state of all cell entry widgets in the `cells` dictionary to 'normal', allowing the user to enter numbers into the cells.

- `readonly_all_cells`: This function sets the state of all cell entry widgets to 'readonly', preventing any further input by setting the cells to read-only.

In [44]:
# Enable user to enter any number
def enable_all_cells():
    for cell in cells.values():
        cell.config(state='normal')
        
# Disable user from entering any number
def readonly_all_cells():
    for cell in cells.values():
        cell.config(state='readonly')

## Populating the Sudoku Grid

In [45]:
def populateGridWithBoard(board):
    global initial_state
    initial_state.clear()
    for i, row in enumerate(board):
        for j, num in enumerate(row):
            cell = cells[(i+2, j+1)]  # Adjusting indices to match the grid layout
            if num != 0:
                cell.insert(0, str(num))
                cell.config(fg="purple")
                cell.config(state='readonly') # Make the cell read-only
                
                initial_state[(i+2, j+1)] = str(num)  # Store the initial number
            else: 
                initial_state[(i+2,j+1)]=""

## Difficulty Level

In [46]:
def easy():
    global difficulty_level
    difficulty_level=SudokuDifficulty.EASY()
    newgame()

def medium():
    global difficulty_level
    difficulty_level=SudokuDifficulty.MEDIUM()
    newgame()

def hard():
    global difficulty_level
    difficulty_level=SudokuDifficulty.HARD()
    newgame()

def expert():
    global difficulty_level
    difficulty_level=SudokuDifficulty.EXPERT()
    newgame()

## Interactive Buttons for Sudoku Solver

These buttons are essential for a user-friendly experience, offering intuitive controls for the game's main features.

In [47]:
# New game button
btn_newgame = tk.Button(root,command=newgame,text="New Game",fg='white',background='black')
btn_newgame.grid(row=1,column=1,columnspan=5,pady=20)

# Solve button
btnsolve=tk.Button(root,command=solve_gui,text="Solve",fg='white',background='black')
btnsolve.grid(row=1,column=5,columnspan=5,pady=20)

# Load the original image using Pillow
ori_image = Image.open("undo.png")
resize_image = ori_image.resize((30, 30), Image.Resampling.LANCZOS)
undo_image = ImageTk.PhotoImage(resize_image)

# Undo button
btn_undo = tk.Button(root, command=undo, image=undo_image,padx=0, pady=0, borderwidth=0,background='pink')
btn_undo.grid(row=17, column=1, columnspan=5)
btn_undo.image = undo_image

# Clear button
btnclear=tk.Button(root,command=clearValues,text="Clear",fg='white',background='#DC3545') # Bootstrap danger red
btnclear.grid(row=17,column=3,columnspan=5)

# Load the original image using Pillow
original_image = Image.open("bulb_icon.png")
resized_image = original_image.resize((30, 30), Image.Resampling.LANCZOS)
bulb_image = ImageTk.PhotoImage(resized_image)

# Hint button
btnhint = tk.Button(root, command=hint, image=bulb_image,padx=0, pady=0, borderwidth=0,background='deepskyblue')
btnhint.grid(row=17, column=5, columnspan=5)
btnhint.image = bulb_image

# Easy button
btneasy=tk.Button(root,command=easy,text="EASY",fg='white',background='deepskyblue')
btneasy.grid(row=18,column=0,columnspan=3,pady=10)

# Medium button
btnmedium=tk.Button(root,command=medium,text="MEDIUM",fg='white',background='dodgerblue')
btnmedium.grid(row=18,column=3,columnspan=3)

# Hard button
btnhard=tk.Button(root,command=hard,text="HARD",fg='white',background='royalblue')
btnhard.grid(row=18,column=6,columnspan=2)

# Expert button
btnexpert=tk.Button(root,command=expert,text="EXPERT",fg='white',background='navy')
btnexpert.grid(row=18,column=8,columnspan=3)

## Starting the Sudoku Game

In [ ]:
difficulty_level=5 # Set the difficulty level of the Sudoku game
newgame() # Initialize a new game with the set difficulty level
root.mainloop() # Start the Tkinter main event loop